
# Practicum II - Movie Database Applicaton
### Collaborators: Abasamia Akpan, Rebecca Brent and Alessia Pizzoccheri

This Notebook provides context, documentation and sample code for a Database Application using IMBd datasets; for clarity, this document has been divided into the following sections:

### 1. Introduction
### 2. Assumptions
### 3. Logical Models
### 4. Database and Relations Definition¶
### 1. Introduction
#### 1.1. Looking at the data
In this section, we analyze and deconstruct the various data sets found on the IMBd page; data is organized in seven TSV files, specifically:

Title AKAs
Title Basics
Title Episodes
Title Ratings
Title Crew
Name Basics
Title Principles Our first step is to establish relationships between each data set and identify a primary key (PK) on each table. Since no information was provided regarding the role of columns, we devised a solution to find unique identifiers within each data set; that is, for each table we compare the total number of rows against the total number of unique values for a given column.
1.2 Establishing PKs
Following the system aforementioned, we identified the following primary keys

In [1]:

import pandas as pd
import numpy as np
import os
import sys
import pip
pip.main(['install','prettytable'])
pip.main(['install','mysql-connector-python'])
pip.main(['install','tqdm'])
pip.main(['install','pymysql'])

from tqdm import tqdm



Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [9]:
import mysql.connector
from prettytable import PrettyTable 
from prettytable import from_csv
from mysql.connector import Error

def create_connection_nodb(host_name, user_name, user_password):
    """ create a connection to mysql host
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_database(connection, db_name):
    """ create a database 
    :param connection: Connection object
    :param db_name: name of database in form of string
    :return:
    """
    cursor = connection.cursor()
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + db_name
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def create_connection(host_name, user_name, user_password, db_name):
    """ create a connection to the (possibly newly created) database
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :param db_name: string name of database 
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        table_name = create_table_sql.split(" ")[5].split("(")[0]
        print("Successfully created table: " + table_name)
    except Error as e:
        print(e)
        
def main():
    
    database = "imbd_application"
    host_name = "localhost"
    user_name = "root"
    user_password = "alupizzo92"
    
    create_table_titlebasics = """CREATE TABLE IF NOT EXISTS titlebasics(
                                    tconst VARCHAR(1000) PRIMARY KEY,
                                    titleType VARCHAR(1000),
                                    primaryTitle VARCHAR(1000),
                                    originalTitle VARCHAR(1000),
                                    isAdult BOOLEAN,
                                    startYear DATE,
                                    endYear DATE,
                                    runTimeMinutes TIME)"""
    
    create_table_titleratings = """CREATE TABLE IF NOT EXISTS titleratings(
                                    tconst VARCHAR(1000) PRIMARY KEY,
                                    averageRating FLOAT,
                                    numVotes INT,
                                    FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""


    create_table_titleepisodes = """CREATE TABLE IF NOT EXISTS titleepisodes(
                                    tconst VARCHAR(1000) PRIMARY KEY,
                                    seasonNumber INT,
                                    episodeNumber INT,
                                    parentTconst VARCHAR(1000),
                                    FOREIGN KEY (parentTconst) REFERENCES titlebasics(tconst))"""

    create_table_titleAKAs = """CREATE TABLE IF NOT EXISTS titleAKAs(
                                AKAsID INT AUTO_INCREMENT PRIMARY KEY,
                                titleID VARCHAR(1000),
                                ordering INT,
                                title VARCHAR(1000),
                                region VARCHAR(1000),
                                language VARCHAR(1000),
                                isoriginaltitle BOOLEAN,
                                FOREIGN KEY (titleID) REFERENCES titlebasics(tconst))"""

    create_table_namebasics = """CREATE TABLE IF NOT EXISTS namebasics(
                                nconst VARCHAR(1000) PRIMARY KEY,
                                primaryname VARCHAR(1000),
                                birthyear DATE,
                                deathyear DATE,
                                noofmovies INT,
                                age INT,
                                currentdate DATE)"""
    
    create_table_principals = """CREATE TABLE IF NOT EXISTS principals(
                                principalsID INT AUTO_INCREMENT PRIMARY KEY,
                                tconst VARCHAR(1000),
                                ordering INT,
                                category VARCHAR(1000),
                                job VARCHAR(1000),
                                nconst VARCHAR(1000),
                                FOREIGN KEY (tconst) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (nconst) REFERENCES namebasics(nconst))"""
    
    create_table_titlecrew = """CREATE TABLE IF NOT EXISTS titlecrew(
                                crewid INT AUTO_INCREMENT PRIMARY KEY,
                                tconst VARCHAR(1000) UNIQUE,
                                FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""
    
    create_table_characters = """CREATE TABLE IF NOT EXISTS characters(
                                characterID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(1000) UNIQUE)"""
    
    create_table_writers = """CREATE TABLE IF NOT EXISTS writers(
                                writerID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(1000) UNIQUE)"""
    
    create_table_directors = """CREATE TABLE IF NOT EXISTS directors(
                                directorID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(1000) UNIQUE)"""
    
    create_table_genres = """CREATE TABLE IF NOT EXISTS genres(
                                genreID INT AUTO_INCREMENT  PRIMARY KEY,
                                genre VARCHAR(1000) UNIQUE)"""
    
    create_table_knownfortitles = """CREATE TABLE IF NOT EXISTS knownfortitles(
                                knownfortitlesID INT AUTO_INCREMENT PRIMARY KEY,
                                titles VARCHAR(1000) UNIQUE)"""
    
    create_table_primaryprofession = """CREATE TABLE IF NOT EXISTS primaryprofession(
                                primaryprofessionID INT AUTO_INCREMENT PRIMARY KEY,
                                profession VARCHAR(1000) UNIQUE)"""
    
    create_table_types = """CREATE TABLE IF NOT EXISTS types(
                                typeID INT AUTO_INCREMENT  PRIMARY KEY,
                                type VARCHAR(1000) UNIQUE)"""
    
    create_table_attributes = """CREATE TABLE IF NOT EXISTS attributes(
                                attributeID INT AUTO_INCREMENT PRIMARY KEY,
                                attribute VARCHAR(1000) UNIQUE)"""
    
    create_table_genretitlebasics = """CREATE TABLE IF NOT EXISTS genretitlebasics(
                                titlebasicsID VARCHAR(1000),
                                genreID INT,
                                PRIMARY KEY (titlebasicsID, genreID),
                                FOREIGN KEY (titlebasicsID) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (genreID) REFERENCES genres(genreID))"""
    
    create_table_crewdirectors = """CREATE TABLE IF NOT EXISTS crewdirectors(
                                crewid INT,
                                directorid INT,
                                PRIMARY KEY (crewid ,directorid),
                                FOREIGN KEY (crewid) REFERENCES titlecrew(crewid),
                                FOREIGN KEY (directorid) REFERENCES directors(directorID))"""
    
    create_table_crewwriters = """CREATE TABLE IF NOT EXISTS crewwriters(
                                crewid INT,
                                writerid INT,
                                PRIMARY KEY (crewid, writerid),
                                FOREIGN KEY (crewid) REFERENCES titlecrew(crewid),
                                FOREIGN KEY (writerid) REFERENCES writers(writerid))"""
    
    create_table_principalscharacters = """CREATE TABLE IF NOT EXISTS principalscharacters(
                                principalsid INT,
                                characterid INT,
                                PRIMARY KEY (principalsid, characterid),
                                FOREIGN KEY (principalsid) REFERENCES principals(principalsID),
                                FOREIGN KEY (characterid) REFERENCES characters(characterid))"""
    
    create_table_AKAattributes = """CREATE TABLE IF NOT EXISTS AKAattributes(
                                akasid INT,
                                attributeid INT,
                                PRIMARY KEY (akasid, attributeid),
                                FOREIGN KEY (akasid) REFERENCES titleAKAs(AKAsID),
                                FOREIGN KEY (attributeid) REFERENCES attributes(attributeid))"""
    
    create_table_AKAtype = """CREATE TABLE IF NOT EXISTS AKAtype(
                                akasid INT,
                                typeid INT,
                                PRIMARY KEY (akasid, typeid),
                                FOREIGN KEY (akasid) REFERENCES titleAKAs(AKAsID),
                                FOREIGN KEY (typeid) REFERENCES type(typeid))"""
    
    create_table_primaryprofessiontitlenames = """CREATE TABLE IF NOT EXISTS primaryprofessiontitlenames(
                                primaryprofessionid INT,
                                namebasicsid VARCHAR(255),
                                PRIMARY KEY (primaryprofessionid, namebasicsid),
                                FOREIGN KEY (primaryprofessionid) REFERENCES primaryprofession(primaryprofessionid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst))"""
    
    create_table_knownfortitlenames = """CREATE TABLE IF NOT EXISTS knownfortitlenames(
                                namebasicsid VARCHAR(255),
                                knownfortitlesid INT,
                                PRIMARY KEY (namebasicsid, knownfortitlesid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst),
                                FOREIGN KEY (knownfortitlesid) REFERENCES knownfortitles(knownfortitlesid))"""
    
    conn = create_connection_nodb(host_name, user_name, user_password)
    
    # create database object if not already created and then connect the cursor to it
    if conn is not None:
        create_database(conn, database)
        conn = create_connection(host_name, user_name, user_password, database)
        
        # create tables
        if conn is not None:
            # create Title Basics table
            create_table(conn, create_table_titlebasics)

            # create Title Ratings table
            create_table(conn, create_table_titleratings)
            
            # create title episodes table
            create_table(conn, create_table_titleepisodes)
            
            # create title AKAs table
            create_table(conn, create_table_titleAKAs)
            
            # create name basics table
            create_table(conn, create_table_namebasics)
            
            # create principals table
            create_table(conn, create_table_principals)
            
            # create title crew table
            create_table(conn, create_table_titlecrew)
            
            # create characters table
            create_table(conn,create_table_characters)
            
            # create writers table
            create_table(conn,create_table_writers)
            
            # create directors table
            create_table(conn,create_table_directors)
            
            # create genres table
            create_table(conn,create_table_genres)
            
            # create known for titles table
            create_table(conn,create_table_knownfortitles)
            
            # create primary profession table
            create_table(conn,create_table_primaryprofession)
            
            # create type table
            create_table(conn,create_table_types)
            
            # create attributes table
            create_table(conn,create_table_attributes)
            
            # create genre title basics table
            create_table(conn, create_table_genretitlebasics)
            
            # create crew directors table
            create_table(conn,create_table_crewdirectors)
            
            # create crew writers table
            create_table(conn,create_table_crewwriters)
            
            # create principals characters table
            create_table(conn, create_table_principalscharacters)
            
            # create AKA Attributes table
            create_table(conn, create_table_AKAattributes)
            
            # create AKA Type table
            create_table(conn, create_table_AKAtype)
            
            # create Primary Profession Title Names table
            create_table(conn, create_table_primaryprofessiontitlenames)
            
            # create known for title names table
            create_table(conn, create_table_knownfortitlenames)
                
        else:
            print("Error! cannot create the database connection.")
    
    else:
        print("Error! Cannot create the MySQL connection.")
    
    


if __name__ == '__main__':
    main()


Connection to MySQL DB successful
Database created successfully
Connection to MySQL DB successful
1071 (42000): Specified key was too long; max key length is 3072 bytes
Successfully created table: titleratings
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
Successfully created table: namebasics
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long; max key length is 3072 bytes
1071 (42000): Specified key was too long;

## SQL communication helper ##

In [10]:
import sqlalchemy
database_username = 'root'
database_password = 'alupizzo92'
database_ip       = 'localhost'
database_name     = 'imbd_application'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [ ]:
## Helper Functions

In [29]:
def load_dataframe(data_frame,table_name):
    data_frame_new=data_frame
    
    data_frame_new.to_sql(table_name,con=database_connection, if_exists='append',index=False)
    
    return 

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

## Title Basics

In [33]:
# read title basic file
title_basics = pd.read_csv('title.basics.tsv', nrows=25, delimiter='\t',encoding='utf-8')
title_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [34]:
def preprocess_dataframe_title_basics(data_frame):
    title_basics_new_series=data_frame

    b = pd.to_datetime(title_basics_new_series['runtimeMinutes'], errors='coerce', format='%M')

    b.to_frame().runtimeMinutes
    df_1 = pd.DataFrame(dict(time_stamps = b)) 

    df_1['runtimeMinutes'] = df_1['time_stamps'].dt.minute
    title_basics_new_series=title_basics_new_series.drop(['runtimeMinutes'],axis=1)
    title_basics_new_series['runtimeMinutes']=df_1['runtimeMinutes']

    #print(title_basics_new_series)
    
    title_basics_new_frame_transpose=title_basics_new_series.astype({'isAdult': 'int32'})
    
    title_basics_new_frame_transpose["isAdult"]=title_basics_new_frame_transpose.isAdult.mask(title_basics_new_frame_transpose.isAdult > 1,1)
    #g.isAdult=g.isAdult.mask(g.isAdult > 1,1)

    #needs to be fixed
    title_basics_new_frame_transpose['startYear']= pd.to_datetime(title_basics.startYear, errors='coerce', format='%Y')

    #title_basics_new_frame_transpose['startYear']=[time.date() for time in title_basics_new_frame_transpose['startYear']]

    
    title_basics_new_frame_transpose['endYear']= pd.to_datetime(title_basics.endYear, errors='coerce', format='%Y')
   # title_basics_new_frame_transpose['endYear']=[time.date() for time in title_basics_new_frame_transpose['endYear']]

    title_basics_new_frame_transpose=title_basics_new_frame_transpose.drop(['genres'],axis=1)
    #print(title_basics_new_frame_transpose)
    
    #title_basics_new_frame_transpose.to_sql('titlebasics',con=database_connection, if_exists='append',index=False)
    new_row = {
        'tconst':None, 
        'titleType':None, 
        'primaryTitle':None,
        'originalTitle':None,
        'isAdult':None, 
        'startYear':None,
        'endYear':None,
        'runtimeMinutes':None         
    } 
  
    title_basics_new_frame_transpose = title_basics_new_frame_transpose.append(new_row, ignore_index=True)
    return title_basics_new_frame_transpose

title_basics_preprocessed=preprocess_dataframe_title_basics(title_basics)

In [12]:
len(title_basics_preprocessed)
title_basics_preprocessed

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes
0,tt0000001,short,Carmencita,Carmencita,0,1894-01-01,NaT,1.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892-01-01,NaT,5.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892-01-01,NaT,4.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892-01-01,NaT,12.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893-01-01,NaT,1.0
...,...,...,...,...,...,...,...,...
7300497,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010-01-01,NaT,NaN
7300498,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010-01-01,NaT,NaN
7300499,tt9916856,short,The Wind,The Wind,0,2015-01-01,NaT,27.0
7300500,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014-01-01,NaT,10.0


In [13]:
title_basics_preprocessed_new=title_basics_preprocessed.rename(columns={"runtimeMinutes": "runTimeMinutes"})

title_basics_preprocessed_new.runTimeMinutes

0           1.0
1           5.0
2           4.0
3          12.0
4           1.0
           ... 
7300497     NaN
7300498     NaN
7300499    27.0
7300500    10.0
7300501     NaN
Name: runTimeMinutes, Length: 7300502, dtype: float64

In [30]:
batch_size = 100000
length = len(title_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_basics_preprocessed, batch_size),total=length):
    load_dataframe(chunk,'titlebasics')

100%|██████████| 74/74 [03:20<00:00,  2.71s/it]


## Title ratings ##

In [24]:
# read data from title ratings file
title_ratings = pd.read_csv('title.ratings.tsv', nrows=25, delimiter='\t',encoding='utf-8')
title_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1656
1,tt0000002,6.1,201
2,tt0000003,6.5,1368
3,tt0000004,6.2,122
4,tt0000005,6.2,2151
5,tt0000006,5.3,115
6,tt0000007,5.4,661
7,tt0000008,5.4,1820
8,tt0000009,5.9,155
9,tt0000010,6.9,6074


In [25]:
def preprocess_dataframe_title_ratings(data_frame):
    title_ratings_new_series=data_frame
    
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'averageRating': 'float'})
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'numVotes': 'int32'})
  
    return title_ratings_new_frame_avg_rating
title_ratings_preprocessed=preprocess_dataframe_title_ratings(title_ratings)

In [26]:
len(title_ratings_preprocessed)

25

In [27]:
batch_size = 100000
length = len(title_ratings_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_ratings_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleratings')

100%|██████████| 1/1 [00:00<00:00, 88.50it/s]


## Title Episodes ##

In [17]:
# read title episodes data
title_episodes = pd.read_csv('title.episode.tsv', nrows=25, delimiter='\t',encoding='utf-8')
title_episodes

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16
5,tt0043693,tt0989125,2,8
6,tt0043710,tt0989125,3,3
7,tt0044093,tt0959862,1,6
8,tt0044668,tt0044243,2,16
9,tt0044901,tt0989125,3,46


In [28]:
def preprocess_dataframe_title_episodes(data_frame):
    title_episodes_new_series=data_frame
    title_episodes_coerce_seasonNum = pd.to_numeric(title_episodes_new_series['seasonNumber'], errors='coerce')
    title_episodes_coerce_episodeNum = pd.to_numeric(title_episodes_new_series['episodeNumber'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_episodes_coerce_seasonNum)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_episodes_coerce_episodeNum)) 

    df_1['seasonNumber'] = df_1['test_column_1']
    df_2['episodeNumber'] = df_2['test_column_2']
    
    title_episodes_new_frame=title_episodes_new_series.drop(['seasonNumber'],axis=1)
    title_episodes_new_frame=title_episodes_new_series.drop(['episodeNumber'],axis=1)
    
    title_episodes_new_frame['seasonNumber']=df_1['seasonNumber']
    title_episodes_new_frame['episodeNumber']=df_2['episodeNumber']
       
    return title_episodes_new_frame

title_episodes_preprocessed = preprocess_dataframe_title_episodes(title_episodes)
title_episodes_preprocessed

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1.0,9.0
1,tt0042816,tt0989125,1.0,17.0
2,tt0042889,tt0989125,NaN,NaN
3,tt0043426,tt0040051,3.0,42.0
4,tt0043631,tt0989125,2.0,16.0
5,tt0043693,tt0989125,2.0,8.0
6,tt0043710,tt0989125,3.0,3.0
7,tt0044093,tt0959862,1.0,6.0
8,tt0044668,tt0044243,2.0,16.0
9,tt0044901,tt0989125,3.0,46.0


In [20]:
title_episodes_preprocessed.tconst = title_episodes_preprocessed.tconst.mask(~title_episodes_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)

In [21]:
batch_size = 100000
length = len(title_episodes_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_episodes_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleepisodes')

100%|██████████| 1/1 [00:00<00:00, 17.65it/s]


## Title Akas ##

In [16]:
title_akas = pd.read_csv('title.akas.tsv', nrows=25, delimiter='\t',encoding='utf-8')
title_akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0
5,tt0000001,6,Carmencita,US,\N,\N,\N,0
6,tt0000001,7,Carmencita,\N,\N,original,\N,1
7,tt0000001,8,カルメンチータ,JP,ja,imdbDisplay,\N,0
8,tt0000002,1,Le clown et ses chiens,\N,\N,original,\N,1
9,tt0000002,2,A bohóc és kutyái,HU,\N,imdbDisplay,\N,0


In [ ]:
def preprocess_dataframe_title_akas(data_frame):
    title_akas_new_series=data_frame
    title_akas_new_series=title_akas_new_series.drop(['types', 'attributes'],axis=1)
    
    title_akas_coerce_ordering = pd.to_numeric(title_akas_new_series['ordering'], errors='coerce')
    title_akas_coerce_isOriginalTitle = pd.to_numeric(title_akas_new_series['isOriginalTitle'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_akas_coerce_ordering)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_akas_coerce_isOriginalTitle)) 

    df_1['ordering'] = df_1['test_column_1']
    df_2['isOriginalTitle'] = df_2['test_column_2']
    
    title_akas_new_frame=title_akas_new_series.drop(['ordering'],axis=1)
    title_akas_new_frame=title_akas_new_series.drop(['isOriginalTitle'],axis=1)
    
    title_akas_new_frame['ordering']=df_1['ordering']
    title_akas_new_frame['isOriginalTitle']=df_2['isOriginalTitle']
    
    
    
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'title': 'float'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'language': 'float'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'region': 'float'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'ordering': 'int32'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'isOriginalTitle': 'int32'})
    


    #print(title_ratings_new_frame_avg_rating)
    #title_ratings_new_frame_avg_rating.to_sql('titleratings',con=database_connection, if_exists='append',index=False)   
    return title_akas_new_frame
title_akas_preprocessed=preprocess_dataframe_title_akas(title_akas)

In [ ]:
title_akas_preprocessed.info()

In [ ]:
title_akas_preprocessed[~title_akas_preprocessed.titleId.isin(title_basics_preprocessed.tconst)]

In [ ]:
title_akas_preprocessed.titleId=title_akas_preprocessed.titleId.mask(~title_akas_preprocessed.titleId.isin(title_basics_preprocessed.tconst),None)

In [ ]:
batch_size=100000
length=len(title_akas_preprocessed)//batch_size+1
for chunk in tqdm(batch(title_akas_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleAKAs')

## Name Basics

In [36]:
name_basics = pd.read_csv('name.basics.tsv', nrows=25, delimiter='\t',encoding='utf-8')
name_basics

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0072308,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0059956,tt0057345,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0078723,tt0077975,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0060827,tt0083922,tt0050976"
5,nm0000006,Ingrid Bergman,1915,1982,"actress,soundtrack,producer","tt0038787,tt0034583,tt0038109,tt0036855"
6,nm0000007,Humphrey Bogart,1899,1957,"actor,soundtrack,producer","tt0037382,tt0043265,tt0034583,tt0040897"
7,nm0000008,Marlon Brando,1924,2004,"actor,soundtrack,director","tt0078788,tt0047296,tt0068646,tt0070849"
8,nm0000009,Richard Burton,1925,1984,"actor,soundtrack,producer","tt0057877,tt0059749,tt0061184,tt0087803"
9,nm0000010,James Cagney,1899,1986,"actor,soundtrack,director","tt0042041,tt0035575,tt0031867,tt0029870"


In [49]:
def preprocess_dataframe_name_basics(data_frame):
    name_basics_new_series=data_frame
    name_basics_new_series=name_basics_new_series.drop(['primaryProfession', 'knownForTitles'],axis=1)
    
    name_basics_new_series['birthYear']= pd.to_datetime(name_basics_new_series.birthYear, errors='coerce', format='%Y')

    #title_basics_new_frame_transpose['startYear']=[time.date() for time in title_basics_new_frame_transpose['startYear']]
    
    name_basics_new_series['deathYear']= pd.to_datetime(name_basics_new_series.deathYear, errors='coerce', format='%Y')
    
    return name_basics_new_series

name_basics_preprocessed = preprocess_dataframe_name_basics(name_basics)

name_basics_preprocessed["noofmovies"] = ""
name_basics_preprocessed["age"] = ""
name_basics_preprocessed["currentdate"] = ""

In [50]:
name_basics_preprocessed_new= pd.to_numeric(name_basics_preprocessed['noofmovies'], errors='coerce')
name_basics_preprocessed_new = pd.to_numeric(name_basics_preprocessed['age'], errors='coerce')
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed.currentdate, errors='coerce', format='%Y')


In [51]:
df_1 = pd.DataFrame(dict(test_column_1 = name_basics_preprocessed_new)) 
df_2 = pd.DataFrame(dict(test_column_2 = name_basics_preprocessed_new)) 

df_1['noofmovies'] = df_1['test_column_1']
df_2['age'] = df_2['test_column_2']

name_basics_preprocessed['noofmovies']=df_1['noofmovies']
name_basics_preprocessed['age']=df_2['age']
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed['currentdate'])

In [52]:
name_basics_preprocessed

,nconst,primaryName,birthYear,deathYear,noofmovies,age,currentdate
0,nm0000001,Fred Astaire,1899-01-01,1987-01-01,NaN,NaN,NaT
1,nm0000002,Lauren Bacall,1924-01-01,2014-01-01,NaN,NaN,NaT
2,nm0000003,Brigitte Bardot,1934-01-01,NaT,NaN,NaN,NaT
3,nm0000004,John Belushi,1949-01-01,1982-01-01,NaN,NaN,NaT
4,nm0000005,Ingmar Bergman,1918-01-01,2007-01-01,NaN,NaN,NaT
5,nm0000006,Ingrid Bergman,1915-01-01,1982-01-01,NaN,NaN,NaT
6,nm0000007,Humphrey Bogart,1899-01-01,1957-01-01,NaN,NaN,NaT
7,nm0000008,Marlon Brando,1924-01-01,2004-01-01,NaN,NaN,NaT
8,nm0000009,Richard Burton,1925-01-01,1984-01-01,NaN,NaN,NaT
9,nm0000010,James Cagney,1899-01-01,1986-01-01,NaN,NaN,NaT


In [53]:
type(list(name_basics_preprocessed.columns[0]))

list

In [54]:
name_basics_preprocessed_new=name_basics_preprocessed.rename(columns={"nconst": "%snconst"})


In [55]:
name_basics_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   nconst       25 non-null     object        
 1   primaryName  25 non-null     object        
 2   birthYear    25 non-null     datetime64[ns]
 3   deathYear    24 non-null     datetime64[ns]
 4   noofmovies   0 non-null      float64       
 5   age          0 non-null      float64       
 6   currentdate  0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), float64(2), object(2)
memory usage: 1.5+ KB


## Approach 1- With sql alchemy ##

In [56]:
batch_size = 100000
length = len(name_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(name_basics_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'namebasics')

100%|██████████| 1/1 [00:00<00:00, 144.48it/s]


In [57]:
# UNCOMMENT TO DISPLAY TABLE ROWS
# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="alupizzo92",
#     database="imbd_application"
# )
# mycursor = mydb.cursor()

# query = "SELECT * FROM namebasics LIMIT 5;"
# mycursor.execute(query)

## Principals-will insert when nconst is in the right format  ##

In [58]:
title_principals = pd.read_csv('title.principals.tsv', nrows=25, delimiter='\t',encoding='utf-8')
title_principals

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
5,tt0000003,1,nm0721526,director,\N,\N
6,tt0000003,2,nm5442194,producer,producer,\N
7,tt0000003,3,nm1335271,composer,\N,\N
8,tt0000003,4,nm5442200,editor,\N,\N
9,tt0000004,1,nm0721526,director,\N,\N


## Title crew ##

In [59]:
title_crew = pd.read_csv('title.crew.tsv', nrows=25, delimiter='\t',encoding='utf-8')
title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
5,tt0000006,nm0005690,\N
6,tt0000007,"nm0374658,nm0005690",\N
7,tt0000008,nm0005690,\N
8,tt0000009,nm0085156,nm0085156
9,tt0000010,nm0525910,\N


In [61]:
title_crew_processed=title_crew.drop(['directors'],axis=1)
title_crew_processed=title_crew.drop(['writers'],axis=1)

In [62]:
batch_size=100000
length=len(title_crew_processed)//batch_size+1

for chunk in tqdm(batch(title_crew_processed,batch_size),total=length):
    load_dataframe(chunk,'titlecrew')

100%|██████████| 1/1 [00:00<00:00, 77.47it/s]


## Writers ##

In [ ]:
def preprocess_dataframe_parsing(data_frame, column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(name=data_frame[column_name].str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    
    return data_frame
writers_preprocessed=preprocess_dataframe_parsing(title_crew.writers, "writers")

In [ ]:
batch_size=100000
length=len(writers_preprocessed)//batch_size+1
for chunk in tqdm(batch(writers_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'writers')

## Directors ##

In [ ]:
directors_preprocessed=preprocess_dataframe_parsing(title_crew.directors, "directors")

In [ ]:
batch_size=100000
length=len(directors_preprocessed)//batch_size+1
for chunk in tqdm(batch(directors_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'directors')

## Genres ##

In [ ]:
genres_preprocessed=preprocess_dataframe_parsing(title_basics.genres, "genres")

In [ ]:
batch_size=100000
length=len(genres_preprocessed)//batch_size+1
for chunk in tqdm(batch(genres_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'genres')

## Known for titles ##

In [ ]:
knownForTitles_preprocessed=preprocess_dataframe_parsing(name_basics.knownForTitles, "knownForTitles")

In [ ]:
batch_size=100000
length=len(knownForTitles_preprocessed)//batch_size+1
for chunk in tqdm(batch(knownForTitles_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'knownForTitles')

## primary profession ##

In [ ]:
pprofession_preprocessed=preprocess_dataframe_parsing(name_basics.primaryProfession, "primaryProfession")

In [ ]:
batch_size=100000
length=len(pprofession_preprocessed)//batch_size+1
for chunk in tqdm(batch(pprofession_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'primaryProfession')

## types ## 

In [ ]:
types_preprocessed=preprocess_dataframe_parsing(title_akas.types, "types")

In [ ]:
batch_size=100000
length=len(types_preprocessed)//batch_size+1
for chunk in tqdm(batch(types_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'types')

## attributes ##

In [ ]:
attributes_preprocessed=preprocess_dataframe_parsing(title_akas.attributes, "attributes")

batch_size=100000
length=len(types_preprocessed)//batch_size+1
for chunk in tqdm(batch(types_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'attributes')

## Characters ##

In [ ]:
def preprocess_dataframe_parsing(data_frame, column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(=data_frame[column_name].str.replace('[\[\]"]',"").str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    
    return data_frame
characters_preprocessed=preprocess_dataframe_parsing(title_principals.characters, "characters")

In [ ]:
batch_size=100000
length=len(characters_preprocessed)//batch_size+1
for chunk in tqdm(batch(directors_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'characters')